In [ ]:
import pandas as pd
import os
from collections import Counter
import nltk
from natsort import natsorted
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize

This code is used to open existing RAW article and filter it. We are removing stop words, and splitting text into list of words.

In [18]:
def articleClean():
    data = open("test_article.txt", "r")
    article = data.read()

    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(article)

    filtered_data = [] 
    for word in word_tokens:
        if word not in stop_words:
            filtered_data.append(word)
    
    string_set = ' '.join(str(e) for e in filtered_data)
    
    file = open("training_set.csv", "w+")
    file.write(string_set)
    file.close()
    
    

Executing articleClean() function.

In [ ]:
articleClean()

In [20]:
all_articles = []
genres = []

Get all files as list

In [21]:
os.chdir("/home/farkhad/Documents/University/Data_Mining/article_classification/articles/technology/training/")
tech_res_art = []
for i in range(1,100):
    tech_res = open("training_tech" + str(i) + ".txt", "r").read()
    all_articles.append(tech_res)
    genres.append("technology")

#PRINT PASS

Get all sport files as list

In [22]:
os.chdir("/home/farkhad/Documents/University/Data_Mining/article_classification/articles/sport/training/")
sport_res_art = []

for i in range(1,100):
    tech_res = open("training_sport" + str(i) + ".txt", "r").read()
    all_articles.append(tech_res)
    genres.append("sport")
#PRINT PASS

Create DataFrame.

In [24]:
d = {'articles' : all_articles, 'genres' : genres}
df = pd.DataFrame(data=d)
#df=df.transpose()
df

,articles,genres
0,The European Commission has clarified claims o...,technology
1,Audio player loading…\n\nAs the European Union...,technology
2,"As mentioned previously, the European Commissi...",technology
3,The European Commission has responded to repor...,technology
4,God of War Ragnarok only released a few days a...,technology
...,...,...
193,Sunday updates will begin when teams start rel...,sport
194,"DALLAS -- Luka Doncic, who admitted fatigue fo...",sport
195,The Portland Trail Blazers finished off their ...,sport
196,Nothing alleviates the hangover from a nasty 0...,sport


Make everything lowercase.

In [25]:
df["articles"] = df["articles"].apply(lambda str : str.lower())
df

,articles,genres
0,the european commission has clarified claims o...,technology
1,audio player loading…\n\nas the european union...,technology
2,"as mentioned previously, the european commissi...",technology
3,the european commission has responded to repor...,technology
4,god of war ragnarok only released a few days a...,technology
...,...,...
193,sunday updates will begin when teams start rel...,sport
194,"dallas -- luka doncic, who admitted fatigue fo...",sport
195,the portland trail blazers finished off their ...,sport
196,nothing alleviates the hangover from a nasty 0...,sport


Calculate the probability.

In [26]:
sum_s = 0
sum_t = 0
for i in df['genres']:
    if i == "sport":
        sum_s +=1
    else:
        sum_t +=1

prob_s = sum_s/len(df)
prob_t = sum_t/len(df)
print(prob_s)
print(prob_t)

0.5
0.5


Remove stopwords and symbols.

In [ ]:
#STOPWORDS CODE HERE

Make a bag of words

In [27]:
df["bow"]  = df["articles"].str.split().apply(Counter)
df

,articles,genres,bow
0,the european commission has clarified claims o...,technology,"{'the': 44, 'european': 4, 'commission': 6, 'h..."
1,audio player loading…\n\nas the european union...,technology,"{'audio': 1, 'player': 1, 'loading…': 1, 'as':..."
2,"as mentioned previously, the european commissi...",technology,"{'as': 4, 'mentioned': 1, 'previously,': 1, 't..."
3,the european commission has responded to repor...,technology,"{'the': 26, 'european': 7, 'commission': 7, 'h..."
4,god of war ragnarok only released a few days a...,technology,"{'god': 4, 'of': 10, 'war': 4, 'ragnarok': 4, ..."
...,...,...,...
193,sunday updates will begin when teams start rel...,sport,"{'sunday': 2, 'updates': 1, 'will': 14, 'begin..."
194,"dallas -- luka doncic, who admitted fatigue fo...",sport,"{'dallas': 3, '--': 3, 'luka': 1, 'doncic,': 3..."
195,the portland trail blazers finished off their ...,sport,"{'the': 73, 'portland': 8, 'trail': 1, 'blazer..."
196,nothing alleviates the hangover from a nasty 0...,sport,"{'nothing': 1, 'alleviates': 1, 'the': 52, 'ha..."


In [38]:
sum_bow_s = {}
sum_bow_t = {}
i=0
for bow in df["bow"]:
    for key in bow:
        if df["genres"][i] == "sport":
            if key in sum_bow_s:
                sum_bow_s[key] += bow[key]
            else:
                sum_bow_s[key] = bow[key]
        else:
            if key in sum_bow_t:
                sum_bow_t[key] +=bow[key]
            else:
                sum_bow_t[key] = bow[key]
    i += 1

#PRINT PASS

calculate total words/terms in Sport and Tech class.

In [43]:
term_occurences_s = 0
term_occurences_t = 0
for key in sum_bow_s:
    term_occurences_s += sum_bow_s[key]
for key in sum_bow_t:    
    term_occurences_t += sum_bow_t[key]

#print(term_occurences_t) 68593
#print(term_occurences_s) 64906

Calculate unique words.

In [47]:
temp = []
unique_term = 0
for bow in df["bow"]:
    for key in bow:
        if key not in temp:
            unique_term += 1
            temp.append(key)

#print(temp)
#print(unique_term) 19813

We store the probability of each word in training data in a dictionary called prob_bow

In [61]:
sum_bow = {}
term_occurences = 0
for bow in df["bow"]:
    for key in bow:
        if key in sum_bow:
            sum_bow[key] += bow[key]
        else:
            sum_bow[key] = bow[key]
        term_occurences += bow[key]

prob_bow = {}
# Adding +1 for better results
for key in sum_bow:
    prob_bow[key] = (sum_bow[key]/term_occurences) + 1

#prob_bow  

Probability of article being Sport or Technology related based on words in it.

In [67]:
def split(words):
    return words.split()

def lower(words):
    return words.lower()

def probSport(article):
    article = split(lower(article))
    likelihood = 1
    evidence = 1
    for term in article:
        if term in sum_bow_s:
            likelihood *= (sum_bow_s[term]+1)/(term_occurences_s+unique_term)
        else:
            likelihood *= 1/(term_occurences_s+unique_term)
        evidence *= prob_bow[term]
    likelihood_prior = likelihood*prob_s
    posterior = likelihood_prior/evidence
    return posterior

def probTech(article):
    article = split(lower(article))
    likelihood = 1
    evidence = 1
    for term in article:
        if term in sum_bow_t:
            likelihood *= (sum_bow_t[term]+1)/(term_occurences_t+unique_term)
        else:
            likelihood *= 1/(term_occurences_t+unique_term)
        evidence *= prob_bow[term]
    likelihood_prior = likelihood*prob_t
    posterior = likelihood_prior/evidence
    return posterior
    
print(probSport("baseball"))
print(probTech("computer smartphone technology"))

6.49156392633727e-05
2.170264505050536e-13
